# DAE Single Shooting Example

From documentation here:
 - https://github.com/casadi/casadi/blob/main/docs/examples/python/dae_single_shooting.py

Compact implementation of a direct single shooting method for DAEs using a minimal
number of CasADi concepts.

Solve the following optimal control problem (OCP) in differential-algebraic equations (DAE):

\begin{equation}
\begin{aligned}
\min_{x_0,x_1,z,u} \quad & \int_{t=0}^{10} x_0^2 + x_1^2 + u^2  dt  \\
\textrm{subject to:} & \\
 & \begin{aligned}
                  \dot{x}_0 &= z x_0 - x_1 + u \quad & \textrm{for} \quad 0 \le t \le 10 \\
                  \dot{x}_1 &= x_0 \quad & \textrm{for} \quad 0 \le t \le 10 \\
                  0 &= x_1^2 + z - 1 \quad & \textrm{for} \quad 0 \le t \le 10 \\
              x_0(t=0) &= 0 & \\
              x_1(t=0) &= 1 & \\
              x_0(t=10) &= 0 & \\
              x_1(t=10) &= 0 & \\
              -0.75 \le u &\le 1 \quad & \textrm{for} \quad 0 \le t \le 10
            \end{aligned}
\end{aligned}
\end{equation}

Note that other methods such as direct collocation or direct multiple shooting
are usually preferably to the direct single shooting method in practise.

In [1]:
import casadi as ca

In [2]:
# Declare variables
n = 2
x = ca.SX.sym("x", n) # Differential states
z = ca.SX.sym("z")   # Algebraic variable
u = ca.SX.sym("u")   # Control

In [3]:
# Differential equation
f_x = ca.vertcat(
    z * x[0] - x[1] + u, 
    x[0]
)

In [4]:
# Algebraic equation
f_z = x[1]**2 + z - 1

In [5]:
# Lagrange cost term (quadrature)
f_q = x[0]**2 + x[1]**2 + u**2

In [6]:
# Create an integrator
dae = {'x': x, 'z': z, 'p': u, 'ode': f_x, 'alg': f_z, 'quad': f_q}

# interval length 0.5s
I = ca.integrator('I', "idas", dae, 0, 0.5)

In [7]:
# All controls
U = ca.MX.sym("U", 20)

In [8]:
# Construct graph of integrator calls
X  = [0, 1]
J = 0
for k in range(20):
    Ik = I(x0=X, p=U[k])
    X = Ik['xf']
    J += Ik['qf']   # Sum up quadratures

In [9]:
# Allocate an NLP solver
nlp = {'x': U, 'f': J, 'g': X}
opts = {"ipopt.linear_solver": "ma27"}
solver = ca.nlpsol("solver", "ipopt", nlp, opts)

In [10]:
# Pass bounds, initial guess and solve NLP
sol = solver(lbx = -0.75, # Lower variable bound
             ubx =  1.0,  # Upper variable bound
             lbg =  0.0,  # Lower constraint bound
             ubg =  0.0,  # Upper constraint bound
             x0  =  0.0) # Initial guess

Exception of type: DYNAMIC_LIBRARY_FAILURE in file "/Users/runner/work/casadi/casadi/build/external_projects/src/ipopt-external/src/Common/IpLibraryLoader.cpp" at line 72:
 Exception message: dlopen(libhsl.dylib, 0x0002): tried: 'libhsl.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibhsl.dylib' (no such file), '/Users/billtubbs/anaconda3/envs/casadi/lib/python3.10/site-packages/casadi/libhsl.dylib' (no such file), '/Users/billtubbs/anaconda3/envs/casadi/bin/../lib/libhsl.dylib' (no such file), '/usr/lib/libhsl.dylib' (no such file, not in dyld cache), 'libhsl.dylib' (no such file), '/usr/local/lib/libhsl.dylib' (no such file), '/usr/lib/libhsl.dylib' (no such file, not in dyld cache)

EXIT: Library loading failure.
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       total  | 705.00us (705.00us) 637.45us (637.45us)         1


In [11]:
sol.keys()

dict_keys(['f', 'g', 'lam_g', 'lam_p', 'lam_x', 'x'])

In [12]:
sol['x']

DM([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])